In [2]:
# import libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime
from sklearn import preprocessing, linear_model, model_selection

%matplotlib inline

Класс моделей ARIMA недостаточно богат для наших данных: с их помощью, например, никак нельзя учесть взаимосвязи между рядами. Это можно сделать с помощью векторной авторегрессии VARIMA, но её питоновская реализация не позволяет использовать регрессионные признаки. Кроме того, авторегрессионный подход не позволяет учитывать, например, взаимодействия между сезонными компонентами. Вы могли заметить, что форма суточных сезонных профилей в будни и выходные немного разная; явно моделировать этот эффект с помощью ARIMA не получится.

Нам нужна более сложная модель. Давайте займёмся сведением задачи массового прогнозирования рядов к регрессионной постановке! Вам понадобится много признаков. Некоторые из них у вас уже есть — это:
<ol>
<li>идентификатор географической зоны
<li>дата и время
<li>количество поездок в периоды, предшествующие прогнозируемому
<li>синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
<li>Кроме того, не спешите выбрасывать построенный вами на прошлой неделе прогнозы — из них может получиться хороший признак для регрессии!
</ol>



Вы можете попробовать разные регрессионный модели, но хорошие результаты, скорее всего, дадут такие, которые будут позволять признакам взаимодействовать друг с другом.

Поскольку прогноз нужен на 6 часов вперёд, проще всего будет построить 6 независимых регрессионных моделей — одна для прогнозирования y^T+1|T, другая для y^T+2|T и т.д.

<ol>Чтобы сдать задание, выполните следующую последовательность действий.
<li>Для каждой из шести задач прогнозирования y^T+i|T,i=1,…,6 сформируйте выборки. Откликом будет yT+i при всевозможных значениях T, а признаки можно использовать следующие:
<ul>
<li>идентификатор географической зоны — категориальный
<li>год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так (done)
<li>синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA (done)
<li>сами значения прогнозов ARIMA y^T+i|TARIMA
<li>количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−K (параметр K можно подбирать; попробуйте начать, например, с 6)
<li>количество поездок из рассматриваемого района в моменты времени yT−24,yT−48,…,yT−24∗Kd (параметр Kd можно подбирать; попробуйте начать, например, с 2)
<li>суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
</ul>
Будьте внимательны при создании признаков — все факторы должны быть рассчитаны без использования информации из будущего: при прогнозировании y^T+i|T,i=1,…,6 вы можете учитывать только значения y до момента времени T включительно.


<li>Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:
Qmay=1R∗739∗6∑r=1R∑T=2016.04.3023:002016.05.3117:00∑i=16y^T|T+ir−yT+ir.
Убедитесь, что ошибка полученных прогнозов, рассчитанная согласно функционалу Q, определённому на прошлой неделе, уменьшилась по сравнению с той, которую вы получили методом индивидуального применения моделей ARIMA. Если этого не произошло, попробуйте улучшить ваши модели.

<li>Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате id,y, где столбец id состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание (1-6); столбец y — ваш прогноз.

<li>Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

<li>Загрузите ноутбук в форму.

Подгружаем данные

In [3]:
# id нужных регионов
regsDf = pd.read_csv('../crowdRegs.csv',names=['id','regId']);  

# времянные ряды для этих регионов
df = pd.read_pickle('../loadData/crowdRegs3.pcl')
regNames = regsDf.regId.values.astype('str')
df.columns = regNames

Наверное, оптимальный способ - пройтись по всем регионам, сформировать требуемую выборку, а потом - состыковать. 
Вероятно, в процессе работы получится векторизовать это действие.
Пожалуй, имеет смысл сначала для всего фрейма добавить общие для всех колонок признаки (тренд, гармоники, даты, дни недели)

In [4]:
def processDataFrame(inpDf, Kw = 5, Ka = 2):
    """
    Обрабатываем сразу весь dateFrame и добавляем признаки, общие для всех рядов
    тренд, гармоники, категориальные перемнные
    для дат, дней недели, etc)

    Parameters:
    Kw number of weeks harmonics
    Ka number of annual harmonics
    """

    inpDf = inpDf.assign(linear = (inpDf.index - datetime.datetime(2014,1,1,0,0,0))/np.timedelta64(1, 'h'))
    
    # час — эти признаки можно пробовать брать и категориальными
    # и непрерывными, можно даже и так, и так

    # добавляем гармонические фичи
    for ind in range(1,Kw+1):
        inpDf['weekCos'+str(ind)]= np.cos(np.pi*inpDf.linear*ind/168)
        inpDf['weekSin'+str(ind)]= np.sin(np.pi*inpDf.linear*ind/168)
     
    for ind in range(1,Ka+1):
        inpDf['yearCos'+str(ind)]= np.cos(2*np.pi*inpDf.linear*ind/8766)        
        inpDf['yearSin'+str(ind)]= np.sin(2*np.pi*inpDf.linear*ind/8766)

    # добавляем числовое и категориальные свойства для дней недели
    inpDf = inpDf.assign(dayOfWeek = inpDf.index.dayofweek)
    lbDays = preprocessing.LabelBinarizer()
    lbDays.fit(list(np.arange(6)))
    DoW = pd.DataFrame(lbDays.transform(inpDf.index.dayofweek),columns = ['dayOfWeek_'+str(x) for x in np.arange(6)],
                       index = inpDf.index)      
    inpDf = inpDf.merge(DoW,left_index=True,right_index=True)

    # добавляем dummy variables для месяца
    inpDf = inpDf.assign(month = inpDf.index.month)
    lbMonths = preprocessing.LabelBinarizer()
    lbMonths.fit(list(np.arange(12)))
    Months = pd.DataFrame(lbMonths.transform(inpDf.index.month),columns = ['month_'+str(x) for x in np.arange(1,13)],
                          index = inpDf.index)      
    inpDf = inpDf.merge(Months,left_index=True,right_index=True);

    # добавляем год (вещественный)
    inpDf = inpDf.assign(year = inpDf.index.year)

    # добавляем день месяца (вещественный)
    inpDf = inpDf.assign(day = inpDf.index.day)

    # добавляем час (вещественный и категориальный)
    inpDf = inpDf.assign(hour = inpDf.index.hour)
    lbHours = preprocessing.LabelBinarizer()
    lbHours.fit(list(np.arange(24)))
    Hours = pd.DataFrame(lbHours.transform(inpDf.index.hour),columns = ['hour_'+str(x) for x in np.arange(24)],
                       index = inpDf.index)      
    inpDf = inpDf.merge(Hours,left_index=True,right_index=True)
    
    return inpDf

Теперь делаем индивидуальную обработку для каждого региона
<ol>
<li> добавляем идентификатор географической зоны — категориальный
<li> количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−K (параметр K можно подбирать; попробуйте начать, например, с 6)
<li> количество поездок из рассматриваемого района в моменты времени yT−24,yT−48,…,yT−24∗Kd (параметр Kd можно подбирать; попробуйте начать, например, с 2)
<li>суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц 
2) 
</ol>

In [5]:
def processSeries(df,tReg,Kh = 6, Kp = 2):
    """
    Обработка одного данного ряда 
    parameters:
        df - начальный датафрейм, из которого выберем для обработки один ряд
        tReg - название ряда, который надо обработать
        Kh - количество отслеживаемых прошлых суточных лагов "назад"
        Kp - количество отслеживаемых прошлых периодических лагов (период 24 часа)

    """
 

    tDf = df.loc[:,tReg.split() + commonFeatures].rename(columns={tReg:'y'})

    tDf = tDf.assign(region = tReg)

    for timeLag in np.arange(1,Kh+1):
        name = 'hourLag_'+str(timeLag)
        tDf.loc[:,name] = tDf.y.shift(periods=timeLag)

    for timeLag in np.arange(1,Kp+1):
        name = 'periodicLag_'+str(timeLag)
        tDf.loc[:,name] = tDf.y.shift(periods=timeLag*24)

    tDf.fillna(0,inplace=True)    

    # суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
    tDf.loc[:,'sum12'] = tDf.y.rolling(window = 12, min_periods = 1).sum()
    tDf.loc[:,'sum24'] = tDf.y.rolling(window = 24, min_periods = 1).sum()
    tDf.loc[:,'sumWeek'] = tDf.y.rolling(window = 168, min_periods = 1).sum()
    tDf.loc[:,'sumMonth'] = tDf.y.rolling(window = 720, min_periods = 1).sum()
    
    #создаём шесть целевые переменных для каждого конца истории
    for targetVar in np.arange(1,7):
        name = 'y'+str(targetVar)
        tDf.loc[:,name] = tDf.y.shift(-targetVar)
    tDf.fillna(0,inplace=True)
    
    return tDf


In [6]:
def getTrips(X):
    model1 = bestModels.get('y1')
    model2 = bestModels.get('y2')
    model3 = bestModels.get('y3')
    model4 = bestModels.get('y4')
    model5 = bestModels.get('y5')
    model6 = bestModels.get('y6')
    pr1 = model1.predict(X)
    pr2 = model2.predict(X)
    pr3 = model3.predict(X)
    pr4 = model4.predict(X)
    pr5 = model5.predict(X)
    pr6 = model6.predict(X)
    pr1[pr1<0] = 0
    pr2[pr2<0] = 0
    pr3[pr3<0] = 0
    pr4[pr4<0] = 0
    pr5[pr5<0] = 0
    pr6[pr6<0] = 0
    
    return np.array([pr1, pr2, pr3, pr4, pr5, pr6])

In [7]:
def saveResults(rdf, fName):
    rnd = np.round

    f = open(fName,'w')
    f.writelines('id,y\n')

    for ind, row in rdf.iterrows():
        historyStart = row.date - datetime.timedelta(hours = 1)

        if historyStart > datetime.datetime(2016,6,30,17):
            continue

        s0 = str(row.region)+'_'+ str(datetime.datetime.strftime(historyStart, "%Y-%m-%d"))+ '_'+ str(historyStart.hour)

        s1 = s0 +'_1,'+str(rnd(row.get('y1'))) + '\n'
        f.writelines(s1)

        s2 = s0 +'_2,'+str(rnd(row.get('y2'))) + '\n'
        f.writelines(s2)

        s3 = s0 +'_3,'+str(rnd(row.get('y3'))) + '\n'
        f.writelines(s3)

        s4 = s0 +'_4,'+str(rnd(row.get('y4'))) + '\n'
        f.writelines(s4)

        s5 = s0 +'_5,'+str(rnd(row.get('y5'))) + '\n'
        f.writelines(s5)

        s6 = s0 +'_6,'+str(rnd(row.get('y6'))) + '\n'
        f.writelines(s6)

    f.close()    

In [8]:
# общая обработка данных
df2 = processDataFrame(df,Kw = 7, Ka = 5)
commonFeatures =  list(set(df2.columns)-set(df.columns.values))
# обработка отдельный рядов
df3 = pd.DataFrame()
for regName in regNames:
    df3 = pd.concat([df3, processSeries(df2,regName,Kh = 12, Kp = 4)])

Разбейте каждую из шести выборок на три части:
<ul>
<li> Обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
<li> Тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
<li> Итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016
</ul>

In [6]:
df3.head()

,y,weekSin1,weekSin2,month,hour_19,hour_9,hour_8,year,hour_5,hour_4,...,sum12,sum24,sumWeek,sumMonth,y1,y2,y3,y4,y5,y6
2014-01-01 00:00:00,87,0.000000,0.000000,1,0,0,0,2014,0,0,...,87.0,87.0,87.0,87.0,92.0,108.0,77.0,47.0,22.0,10.0
2014-01-01 01:00:00,92,0.018699,0.037391,1,0,0,0,2014,0,0,...,179.0,179.0,179.0,179.0,108.0,77.0,47.0,22.0,10.0,18.0
2014-01-01 02:00:00,108,0.037391,0.074730,1,0,0,0,2014,0,0,...,287.0,287.0,287.0,287.0,77.0,47.0,22.0,10.0,18.0,19.0
2014-01-01 03:00:00,77,0.056070,0.111964,1,0,0,0,2014,0,0,...,364.0,364.0,364.0,364.0,47.0,22.0,10.0,18.0,19.0,28.0
2014-01-01 04:00:00,47,0.074730,0.149042,1,0,0,0,2014,0,1,...,411.0,411.0,411.0,411.0,22.0,10.0,18.0,19.0,28.0,37.0


In [9]:
regDf = df3.get('region')
regDf = regDf.reset_index()
regDf.rename(columns={'index':'date'},inplace=True)
regDf.head()

,date,region
0,2014-01-01 00:00:00,1075
1,2014-01-01 01:00:00,1075
2,2014-01-01 02:00:00,1075
3,2014-01-01 03:00:00,1075
4,2014-01-01 04:00:00,1075


In [8]:
df3.reset_index(inplace=True)
df3 = pd.get_dummies(df3,'region')
print df3.shape

(2232576, 202)


In [10]:
startTrain = '2015-01-01 00:00:00'
endTrain   = '2016-04-30 23:00:00'

startValidation = '2016-05-01 00:00:00'
endValidation   = '2016-05-31 23:00:00'

startTest = '2016-06-01 00:00:00'
endTest   = '2016-06-30 23:00:00'

In [11]:
trainSet = df3.query('index >= @startTrain and index <= @endTrain') 
validationSet = df3.query('index >= @startValidation and index <= @endValidation')
testSet = df3.query('index >= @startTest and index <= @endTest')

Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:
<ul>
<li>Допускала попарные взаимодействия между признаками
<li>Была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы)
</ul>

Попробуем обучить Ridge классификатор с L2 регуляризацией.

In [12]:
targetList = ['y1','y2','y3','y4','y5','y6']
paramsAlpha = np.linspace(0.1,1,5)

In [38]:
bestModels = dict()
for target in targetList:
    print target
    dropCols = [x for x in targetList if x != target ]
    dropCols.append('y')
    dropCols.append('index')
    tmpTrain = trainSet.drop(dropCols,axis = 1)
    tmpTrain.rename(columns={target:'y'},inplace=True)
    
    tmpValidation = validationSet.drop(dropCols,axis = 1)
    tmpValidation.rename(columns={target:'y'},inplace=True)
    
    minErr = np.inf
    bestAlpha = 0
    
    
    for a in paramsAlpha:
        regressor = linear_model.Ridge(alpha= a)
        regressor.fit(tmpTrain.drop('y',axis = 1),tmpTrain.loc[:,'y'])
        prediction = regressor.predict(tmpValidation.drop('y',axis = 1))
        err = np.abs(prediction - tmpValidation.loc[:,'y'])
        err = err.mean()
        print a, err
        if err <minErr:
            minErr = err
            bestAlpha = a
            bestModel = regressor
    
    bestModels.update({target:bestModel})
            
    print 'Smallest error {:2.3f} at a = {:2.3f}'.format(err,a)     

y1
0.1 34.7574038082
0.325 46.2909079751
0.55 50.034249977
0.775 51.5518956719
1.0 52.1829291915
Smallest error 52.183 at a = 1.000
y2
0.1 55.5007634777
0.325 85.52214317
0.55 95.2823145344
0.775 99.3390826059
1.0 101.127500322
Smallest error 101.128 at a = 1.000
y3
0.1 71.002708543
0.325 119.801945082
0.55 135.497596194
0.775 142.016950877
1.0 144.907194319
Smallest error 144.907 at a = 1.000
y4
0.1 78.5565011345
0.325 145.904162356
0.55 167.464878144
0.775 176.47496423
1.0 180.521965562
Smallest error 180.522 at a = 1.000
y5
0.1 83.4584632643
0.325 167.314149492
0.55 193.995328347
0.775 205.161459102
1.0 210.208393218
Smallest error 210.208 at a = 1.000
y6
0.1 87.3599701127
0.325 183.776210666
0.55 214.251556546
0.775 226.996624755
1.0 232.76826307
Smallest error 232.768 at a = 1.000


In [13]:
# now there are three regressors. WE have to use them.
#np.save('bestModels.npy',bestModels)
#bestModels = np.load('bestModels.npy').item()

In [ ]:
bestModels = dict()

from sklearn.tree import DecisionTreeRegressor as DTR

for target in targetList:
    print target
    dropCols = [x for x in targetList if x != target ]
    dropCols.append('y')
    dropCols.append('index')
    tmpTrain = trainSet.drop(dropCols,axis = 1)
    tmpTrain.rename(columns={target:'y'},inplace=True)
    
    tmpValidation = validationSet.drop(dropCols,axis = 1)
    tmpValidation.rename(columns={target:'y'},inplace=True)
    
    regressor = DTR()
    regressor.fit(tmpTrain.drop('y',axis = 1),tmpTrain.loc[:,'y'])
    prediction = regressor.predict(tmpValidation.drop('y',axis = 1))
    err = np.abs(prediction - tmpValidation.loc[:,'y'])
    err = err.mean()

    bestModels.update({target:regressor})
            
    print 'Smallest error {:2.3f} at a = {:2.3f}'.format(err,a)     

y1
Smallest error 24.727 at a = 1.000
y2
Smallest error 25.598 at a = 1.000
y3
Smallest error 26.208 at a = 1.000
y4
Smallest error 26.960 at a = 1.000
y5
Smallest error 26.973 at a = 1.000
y6


In [66]:
dropCols = ['index','y','y1','y2','y3','y4','y5','y6']
prediction = getTrips(testSet.drop(dropCols, axis = 1))

predictionDf = pd.DataFrame(prediction.T,columns=['y1','y2','y3','y4','y5','y6'])
predictionDf.set_index(testSet.index,inplace=True)
predictionDf = predictionDf.merge(regDf,left_index=True,right_index=True,how='left')
predictionDf = predictionDf.round()

diff  = np.abs(predictionDf.y1-testSet.y1)+np.abs(predictionDf.y2-testSet.y2)+np.abs(predictionDf.y3-testSet.y3)+np.abs(predictionDf.y4-testSet.y4)+np.abs(predictionDf.y5-testSet.y5)+np.abs(predictionDf.y6-testSet.y6)
print 'Error is', diff.mean()/6

Error is 30.2730187908


In [67]:
# теперь надо сохранить это в файл
fName = 'res_week5-4.csv'
saveResults(predictionDf,fName)

In [68]:
predictionDf

,y1,y2,y3,y4,y5,y6,date,region
21168,6.0,8.0,5.0,3.0,8.0,21.0,2016-06-01 00:00:00,1075
21169,9.0,4.0,10.0,8.0,28.0,14.0,2016-06-01 01:00:00,1075
21170,5.0,2.0,8.0,16.0,53.0,98.0,2016-06-01 02:00:00,1075
21171,2.0,8.0,17.0,39.0,87.0,99.0,2016-06-01 03:00:00,1075
21172,7.0,17.0,61.0,87.0,49.0,99.0,2016-06-01 04:00:00,1075
21173,21.0,41.0,87.0,67.0,60.0,86.0,2016-06-01 05:00:00,1075
21174,38.0,86.0,67.0,62.0,82.0,120.0,2016-06-01 06:00:00,1075
21175,79.0,59.0,129.0,83.0,66.0,120.0,2016-06-01 07:00:00,1075
21176,112.0,77.0,70.0,83.0,120.0,114.0,2016-06-01 08:00:00,1075
21177,62.0,56.0,114.0,84.0,120.0,114.0,2016-06-01 09:00:00,1075


In [20]:
testSet.query("region == '2168'")

,y,weekSin1,weekSin2,month,hour_19,hour_9,hour_8,year,hour_5,hour_4,...,sum12,sum24,sumWeek,sumMonth,y1,y2,y3,y4,y5,y6
2016-06-01 00:00:00,85,-5.095769e-14,-1.019154e-13,6,0,0,0,2016,0,0,...,1251.0,2117.0,11074.0,48866.0,42.0,10.0,0.0,1.0,59.0,82.0
2016-06-01 01:00:00,42,1.869887e-02,3.739119e-02,6,0,0,0,2016,0,0,...,1172.0,2084.0,11074.0,48809.0,10.0,0.0,1.0,59.0,82.0,63.0
2016-06-01 02:00:00,10,3.739119e-02,7.473009e-02,6,0,0,0,2016,0,0,...,1108.0,2094.0,11084.0,48783.0,0.0,1.0,59.0,82.0,63.0,117.0
2016-06-01 03:00:00,0,5.607045e-02,1.119645e-01,6,0,0,0,2016,0,0,...,1032.0,2094.0,11084.0,48783.0,1.0,59.0,82.0,63.0,117.0,54.0
2016-06-01 04:00:00,1,7.473009e-02,1.490423e-01,6,0,0,0,2016,0,1,...,911.0,2092.0,11082.0,48775.0,59.0,82.0,63.0,117.0,54.0,58.0
2016-06-01 05:00:00,59,9.336361e-02,1.859116e-01,6,0,0,0,2016,1,0,...,877.0,2062.0,11101.0,48768.0,82.0,63.0,117.0,54.0,58.0,29.0
2016-06-01 06:00:00,82,1.119645e-01,2.225209e-01,6,0,0,0,2016,0,0,...,878.0,2025.0,11128.0,48727.0,63.0,117.0,54.0,58.0,29.0,64.0
2016-06-01 07:00:00,63,1.305262e-01,2.588190e-01,6,0,0,0,2016,0,0,...,838.0,1972.0,11118.0,48650.0,117.0,54.0,58.0,29.0,64.0,96.0
2016-06-01 08:00:00,117,1.490423e-01,2.947552e-01,6,0,0,1,2016,0,0,...,844.0,1947.0,11182.0,48622.0,54.0,58.0,29.0,64.0,96.0,72.0
2016-06-01 09:00:00,54,1.675062e-01,3.302791e-01,6,0,1,0,2016,0,0,...,768.0,1899.0,11204.0,48553.0,58.0,29.0,64.0,96.0,72.0,62.0
